In [ ]:
import pandas as pd
import numpy as np
from itertools import product
import os
import matplotlib.pyplot as plt

In [ ]:
path = '/home/trungphong/Desktop/AIS/folder_Data/Data/'

In [ ]:
df = pd.read_csv(path+'REE.csv')

In [ ]:
data = np.flip(np.array(df[['CLOSE','OPEN','HIGH','LOW','VOLUME']]),axis =0)
training = data

In [ ]:

# TP calculation function

def TP_(training):
    TP = np.zeros((len(training),), dtype=float)
    for i in range(len(training)) :
        TP[i] = (training[i][0]+training[i][1]+training[i][2])/3
    return TP

# MF calculation function

def MF_(TP, V_):
    MF = np.zeros((len(training),), dtype=float)
    for i in range(len(TP)):
        MF[i]=TP[i]*V_[i]
    return MF

# PMF and NMF calculation function

def cal_PMF_NMF(MF):
    PMF_NMF = np.zeros((len(training),), dtype=tuple)
    PMF_NMF[0] = (MF[0],0)
    for i in range(1,len(PMF_NMF)):
        if MF[i]<MF[i-1]:
            PMF_NMF[i]=(0,MF[i])
            continue
        PMF_NMF[i]=(MF[i],0)
    return PMF_NMF

# MFI calculation function

def calcu_MFI(PMF_NMF):
    MFI = np.zeros((len(training)-12,), dtype=float)
    PMF_,NMF_=0,0
    for iter_ in range(0,13):
        PMF_+=PMF_NMF[iter_][0]
        NMF_+=PMF_NMF[iter_][1]
    MFI[0]=PMF_*100/(PMF_+NMF_)
    for step_ in range(1,len(MFI)):
        PMF_= PMF_- PMF_NMF[step_-1][0] + PMF_NMF[step_+12][0]
        NMF_= NMF_- PMF_NMF[step_-1][1] + PMF_NMF[step_+12][1]
        MFI[step_]=PMF_*100/(PMF_+NMF_)
    return MFI

In [ ]:
V_ = training[:,4] # volume 
TP = TP_(training) # call calculator TP
MF = MF_(TP, V_) # call calculator MF
PMF_NMF = cal_PMF_NMF(MF) # call calculator PMF and NMF
MFI = calcu_MFI(PMF_NMF) # call calculator MFI(13)
MFI = MFI[1:] #pass 13 first days 

In [ ]:
# create label for MFI
label = np.zeros((len(training)-13,), dtype=bool)
for iter_ in range(13,len(training)):
    label[iter_-13]=training[iter_][0]>=training[iter_-1][0]

In [ ]:
df = pd.DataFrame({'MFI':MFI,'Label':label})
df.to_csv('REE_MFI.csv', index=True)